# Neuropixels data conversion to NWB

This notebook demonstrates how to convert an example session from the Neuropixels dataset to NWB.

## Neuropixels dataset

This dataset contains in vivo extracellular electrophysiology recordings from rats performing a value-based decision-making task. Neural data were acquired using Neuropixels probes (384 channels, 30 kHz sampling rate) with Neuropix-PXI hardware and OpenEphys, and preprocessed using Kilosort 2.5 with manual curation in Phy. Trials were initiated by a nose-poke in a lit center port and required maintaining a center fixation for 0.8 to 1.2 seconds, during which a tone indicated the possible reward size. A subsequent side LED indicated the potential reward location, followed by a delay period drawn from an exponential distribution (mean = 2.5 s). Rats could opt out at any time by poking the unlit port, restarting the trial. Catch trials, where the delay period only ended if the rat opted out, constituted 15-25% of the trials. Rats received penalties for premature fixation breaks. Additionally, the tasks introduced semi-observable hidden states by varying reward statistics across uncued blocks (high, low, and mixed), structured hierarchically, with blocks transitioning after 40 successfully completed trials.
This notebook demonstrates how to convert an example session to NWB.

This dataset have the following data streams:
- Behavior: Bpod output (.mat)
- Recording AP, LFP: OpenEphys (binary format)
- Units: Phy output

## Notes on the conversion

The conversion notes is located in `src/constantinople_lab_to_nwb/schierek_embargo_2024/schierek_embargo_2024_notes.md`. This file contains information about the expected file structure and the conversion process.

## Running the conversion

To run a specific conversion, you might need to install first some conversion specific dependencies that are located in each conversion directory:

In [3]:
#!pip install -r ../schierek_embargo_2024_requirements.txt

## Convert a single session to NWB

The `schierek_embargo_2024_convert_session.py` script defines the `session_to_nwb` function that converts a session of Neuropixels data to NWB.

In [2]:
from constantinople_lab_to_nwb.schierek_embargo_2024.schierek_embargo_2024_convert_session import session_to_nwb

# Parameters for conversion

# The path to the NWB file to write.
nwbfile_path = "/Volumes/T9/Constantinople/nwbfiles/sub-J076_ephys.nwb"
# The OpenEphys recording folder path (make sure to include the 'Record Node #' in the folder path)
folder_path = "/Volumes/T9/Constantinople/Ephys Data/J076_2023-12-12_14-52-04/Record Node 117"
# The name of the *raw* recording stream (e.g. )
raw_stream_name = "Neuropix-PXI-119.ProbeA-AP"
# The name of the *LFP* recording stream (e.g. )
lfp_stream_name = "Neuropix-PXI-119.ProbeA-LFP"
# The path to the processed spike sorting file (.mat). This file contains the "SU" named struct that contains the processed spike data.
# This file also contains the "S" named struct containing the processed behavior data.
spike_sorting_mat_file_path = "/Volumes/T9/Constantinople/Ephys Data/J076_2023-12-12.mat"
# The path to the raw Bpod data
bpod_file_path = "/Volumes/T9/Constantinople/raw_Bpod/J076/DataFiles/J076_RWTautowait2_20231212_145250.mat"

# Task specific parameters

# The column name mapping is used to rename the columns in the processed data to more descriptive column names.
# New (unseen) task parameters should be added here.
column_name_mapping = dict(
    NoseInCenter="nose_in_center",
    TrainingStage="training_stage",
    Block="block_type",
    BlockLengthAd="num_trials_in_adaptation_blocks",
    BlockLengthTest="num_trials_in_test_blocks",
    ProbCatch="catch_percentage",
    RewardDelay="reward_delay",
    RewardAmount="reward_volume_ul",
    WaitForPoke="wait_for_center_poke",
    hits="is_rewarded",
    vios="is_violation",
    optout="is_opt_out",
    wait_time="wait_time",
    wait_thresh="wait_time_threshold",
    wait_for_cpoke="wait_for_center_poke",
    zwait_for_cpoke="z_scored_wait_for_center_poke",
    RewardedSide="rewarded_port",
    Cled="center_poke_times",
    Lled="left_poke_times",
    Rled="right_poke_times",
    l_opt="left_opt_out_times",
    r_opt="right_opt_out_times",
    ReactionTime="reaction_time",
    slrt="short_latency_reaction_time",
    iti="inter_trial_interval",
)
# The column descriptions are used to add descriptions to the columns in the processed data.
# New (unseen) task parameter descriptions should be added here.
column_descriptions = dict(
    NoseInCenter="The time in seconds when the animal is required to maintain center port to initiate the trial (uniformly drawn from 0.8 - 1.2 seconds).",
    TrainingStage="The stage of the training.",
    Block="The block type (High, Low or Test). High and Low blocks are high reward (20, 40, or 80μL) or low reward (5, 10, or 20μL) blocks. Test blocks are mixed blocks.",
    BlockLengthAd="The number of trials in each high reward (20, 40, or 80μL) or low reward (5, 10, or 20μL) blocks.",
    BlockLengthTest="The number of trials in each mixed blocks.",
    ProbCatch="The percentage of catch trials.",
    RewardDelay="The delay in seconds to receive reward, drawn from exponential distribution with mean = 2.5 seconds.",
    RewardAmount="The volume of reward in microliters.",
    hits="Whether the subject received reward for each trial.",
    vios="Whether the subject violated the trial by not maintaining center poke for the time required by 'nose_in_center'.",
    optout="Whether the subject opted out for each trial.",
    WaitForPoke="The time (s) between side port poke and center poke.",
    wait_time="The wait time for the subject for for each trial in seconds, after removing outliers."
        " For hit trials (when reward was delivered) the wait time is equal to the reward delay."
        " For opt-out trials, the wait time is equal to the time waited from trial start to opting out.",
    wait_for_cpoke="The time between side port poke and center poke in seconds, includes the time when the subject is consuming the reward.",
    zwait_for_cpoke="The z-scored wait_for_cpoke using all trials.",
    RewardedSide="The rewarded port (Left or Right) for each trial.",
    Cled="The time of center port LED on/off for each trial (2 x ntrials).",
    Lled="The time of left port LED on/off for each trial (2 x ntrials).",
    Rled="The time of right port LED on/off for each trial (2 x ntrials).",
    l_opt="The time of left port entered/exited for each trial (2 x ntrials).",
    r_opt="The time of right port entered/exited for each trial (2 x ntrials).",
    ReactionTime="The reaction time in seconds.",
    slrt="The short-latency reaction time in seconds.",
    iti="The time to initiate trial in seconds (the time between the end of the consummatory period and the time to initiate the next trial).",
    wait_thresh="The threshold in seconds to remove wait-times (mean + 1*std of all cumulative wait-times).",
)

# Optional parameters

# Whether to run a stub test conversion. The stubbed file will only contain a small portion of data.
# When running the full conversion stub_test should be disabled.
stub_test = True
# Whether to overwrite an existing NWB file.
overwrite = True

# Run the `session_to_nwb` function with the parameters to convert to NWB
session_to_nwb(
    nwbfile_path=nwbfile_path,
    openephys_recording_folder_path=folder_path,
    ap_stream_name=raw_stream_name,
    lfp_stream_name=lfp_stream_name,
    processed_spike_sorting_file_path=spike_sorting_mat_file_path,
    raw_behavior_file_path=bpod_file_path,
    column_name_mapping=column_name_mapping,
    column_descriptions=column_descriptions,
    stub_test=stub_test,
    overwrite=overwrite,
)